In [1]:
from tensorflow import keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import copy
import numpy as np
import dlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
   tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
model_enged  = keras.models.load_model('C:/Users/Moh.Massoud/ML/Engagment level/DAiSEE/Models/engagement86.h5')
model_bored  = keras.models.load_model('C:/Users/Moh.Massoud/ML/Engagment level/DAiSEE/Models/boredom.h5')
model_conf   = keras.models.load_model('C:/Users/Moh.Massoud/ML/Engagment level/DAiSEE/confusion.h5')
model_frus   = keras.models.load_model('C:/Users/Moh.Massoud/ML/Engagment level/DAiSEE/Frustration.h5')

In [4]:
def combine_horizontally(images,padding=0):
    max_height = 0  # find the max height of all the images
    total_width = 0  # the total width of the images (horizontal stacking)
    for img in images:
        # open all images and find their sizes
        image_height = img.shape[0]
        image_width = img.shape[1]
        if image_height > max_height:
            max_height = image_height
        # add all the images widths
        total_width += image_width
    # create a new array with a size large enough to contain all the images
    # also add padding size for all the images except the last one
    final_image = np.ones((max_height, (len(images)-1)*padding+ total_width, 3), dtype=np.uint8)
    current_x = 0  # keep track of where your current image was last placed in the x coordinate
    for image in images:
        # add an image to the final array and increment the x coordinate
        height = image.shape[0]
        width = image.shape[1]
        final_image[:height,current_x :width+current_x, :] = image
        #add the padding between the images
        current_x += width+padding
    return final_image

In [7]:
def addBar2Frame(predictions, labels):
    plt.bar(labels, predictions)
    plt.xticks(rotation=16)
    plt.style.use('dark_background')
#     plt.yticks([],[])
    plt.ylim([0,1])
    plt.autoscale(enable=False, axis='y')
    plt.savefig('temp.png')
    mat = cv2.imread('temp.png')
    plt.clf()
    return mat

In [17]:
cap = cv2.VideoCapture(1) # Check if the webcam is opened correctLy 
if not cap.isOpened():
    cap = cv2.VideoCapture('C:/Users/Moh.Massoud/ML/Engagment level/DAiSEE/Demos/demo(hafez).mp4') 
if not cap.isOpened():
    raise IOError("Cannot open webcam")
    
    
frame_width = 1152
frame_height = 576

size = (frame_width, frame_height)
result = cv2.VideoWriter('demo(hafez).avi', 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         25, size)

detector = dlib.get_frontal_face_detector()
font = cv2.FONT_HERSHEY_SIMPLEX

frame_count = 0
state = 0
printState = -1
count = 0
while True:
    _, frame = cap.read()
    count += 1
    if _:
        frame_count += 1
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = detector(gray)
        #predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

        engage=0
        roi=[]
        for face in faces:
            x1 = face.left()
            y1 = face.top()
            x2 = face.right()
            y2 = face.bottom()
            # width = x2 - x1
            # height = y2 - y1
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            if len(frame[y1:y2, x1:x2]) <= 0:
                continue
            roi.append(cv2.resize(cv2.cvtColor(frame[y1:y2, x1:x2], cv2.COLOR_BGR2GRAY), (48,48)))
            predictions_engage = []
            predictions_board = []
            prediction_frus = []
            prediction_conf = []
            if(len(roi)>0):
                test_images = np.expand_dims(roi, axis=3)
                predictions_engage = model_enged.predict(test_images)
                predictions_board = model_bored.predict(test_images)
                predictions_frus = model_frus.predict_on_batch(test_images)
                predictions_conf = model_conf.predict_on_batch(test_images)
               
            predict = 0        
            for i in range(len(predictions_engage)):
                predict=np.argmax(predictions_engage[i])
            state += predict

            if frame_count > 25:
                printState = round(state/25,1)
                state = 0
                frame_count = 0

            if printState != -1 and len(roi):    
                cv2.putText(frame, f"{'Engaged' if printState else 'Not Engaged'}", (x1, y1), font, 0.5, (0, 0, 255), 2, cv2.LINE_4)
            predictions = []
            predictions.extend(predictions_engage[0])
            predictions.extend(predictions_conf[0])
            predictions.extend(predictions_board[0])
            
            predictions.extend(predictions_frus[0])
            
            predictions_labels = ['not engaged', 'engaged', 'not confused', 'confused', 'not board', 'board', 'not frustrated', 'frustrated']
#             print(predictions, len(predictions))
#             print(predictions_labels, len(predictions_labels))
            mat = addBar2Frame(predictions, predictions_labels)
            
            frame = combine_horizontally([frame,mat])
#             print(frame.shape)
            cv2.imshow("Drowsiness detection", frame)
            result.write(frame)
    else:
        cap. release() 
        cv2.destroyAllWindows() 
        break
    if cv2.waitKey(2) & 0xFF == ord('q'): 
        cap. release() 
        cv2.destroyAllWindows() 
        break

<Figure size 432x288 with 0 Axes>